first create the schema and load data

In [0]:
%sql
create catalog if not exists catalog1;
create database if not exists catalog1.schema1;
create volume if not exists catalog1.schema1.volume1;



In [0]:

dbutils.fs.mkdirs("/Volumes/catalog1/schema1/volume1/directory1/")

In [0]:

from pyspark.sql.session import SparkSession
print(spark)#already instantiated by databricks
spark1=SparkSession.builder.getOrCreate()
print(spark1)#we instantiated

In [0]:

csv_df1=spark.read.csv("dbfs:///Volumes/catalog1/schema1/volume1/directory1/patients.csv")
#csv_df1.show(2)#display with produce output in a dataframe format
#print(csv_df1.printSchema())
display(csv_df1)#display with produce output in a beautified table format, specific to databricks

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.ingestion_volume.source;

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.ingestion_volume.target;

#1.Header Concepts </br>
 if we give header= ture we, are asking spark to take the first row as header and not as a data

In [0]:
csv_df1=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs_header",header=True)
#print(csv_df1.printSchema())
csv_df1.show(2)

By default it will use _c0,_c1..._cn it will apply as column headers,

In [0]:
csv_df2=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs")
display(csv_df2.show(2))

#2.ToDF
 we use toDF(colnames) we can define our own headers..

In [0]:
csv_df2=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs").toDF("ID","FNAME","LNAME","AGE","PROF")
display(csv_df2.show(2))

comparing both teh tables with header and with TODF header

In [0]:
csv_df1.printSchema()
csv_df2.printSchema()

#3. Inferring Schema 
 Performance Consideration: Use this function causiously because it scans the entire data by immediately evaluating and executing
hence, not good for large data or not good to use on the predefined schema dataset

In [0]:
csv_df1=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs",inferSchema =True,samplingRatio=.10).toDF("ID","FNAME","LNAME","AGE","PROF")
csv_df1.where("id in (4000010,4000014)").show(2)
#csv_df1.printSchema()
#display(csv_df1)

#4.seperator
Using delimiter or seperator option

In [0]:
csv_df2 = spark.read.csv("dbfs:///Volumes/workspace/ingestion_volume/source/custs_header",header=True, sep=',')
csv_df2.show(2)
csv_df2.printSchema()

#5. Using different options to create dataframe with csv and other module... 

In [0]:
csv_df1=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs_header",inferSchema=True,header=True,sep=',')
csv_df1.show(2)

or another way of creating dataframe (from any sources whether builtin or external)...
option can be used for 1 or 2 option...

In [0]:
csv_df2=spark.read.option("header","True").option("inferSchema","true").option("sep",",").format("csv").load("/Volumes/workspace/ingestion_volume/source/custs_header")
csv_df2.show(2)

#option can be used for multiple options in one function as a parameter...

In [0]:
csv_df3=spark.read.options(header="True",inferSchema="true",sep=",").format("csv").load("/Volumes/workspace/ingestion_volume/source/custs_header")
csv_df3.show(2)

Reading data from multiple files & Multiple Path (We still have few more options)

In [0]:
csv_df1=spark.read.csv("/Volumes/workspace/ingestion_volume/source/custs*",inferSchema=True,header=True,sep=',') 
print(csv_df1.count())

In [0]:

csv_df1=spark.read.csv(path=["/Volumes/workspace/ingestion_volume/source/custs*","/Volumes/workspace/ingestion_volume/source/custs_header"],inferSchema=True,header=True,sep=',')
print(csv_df1.count())